In [47]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

import pywikibot as pb
from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote

from io import BytesIO
import re

import dateparser as dp
import datetime
import shutil
from hashlib import sha1

commons_site = pb.Site("commons", "commons")

In [48]:
def is_commons_file (sha):
    pages = pb.site.APISite.allimages(commons_site, sha1=sha)
    result = False
    for page in pages :
        print (page)
        result = True
        break
    return result
    
def get_hash (file_path):
    image_file = open(file_path, "rb")
    byte_stream = BytesIO(image_file.read()).getvalue()
    image_file.close()
    hashObject = sha1()
    hashObject.update(byte_stream)
    return hashObject.hexdigest()

In [49]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [50]:
#### User input:
url = 'http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/consejoministros/Paginas/2018/080618cministros.aspx'

#### User input:
categories = ['Palacio de la Moncloa', 
              'June 2018 in Madrid',
              'Pedro Sánchez Pérez-Castejón in 2018']

In [51]:
while True:
    request = requests.get(url)
    if request is not None:
        #print("{0}".format(u''.join(request.text).encode('utf-8')))
        response = request.text
        break
    else :
        print ('Trying')

soup = BeautifulSoup(response, 'html.parser')

In [52]:
title = soup.find_all("h1", class_="contenidoTitulo")[0].get_text().strip().replace(':', ' - ').replace('  ', ' ')
#### User input:
#title = 'Imposición del Toisón de Oro a la princesa de Asturias'
title

'Gobierno de Pedro Sánchez en la XII Legislatura'

In [53]:
try:
    date_string = soup.find_all("div", class_="contenidoFecha")[0].get_text()
    pub_date_sp = ' '.join(date_string.strip().split(' ')[-5:])
    print (pub_date_sp)
    parsed_pub_date = dp.parse(pub_date_sp)
    pub_date = parsed_pub_date.strftime('%Y-%m-%d')
    
    year_date = parsed_pub_date.strftime('%Y')
    if 'La Moncloa' in date_string:
        categories.append('Palacio de la Moncloa')
        categories.append(year_date + ' in Madrid')
except :
    pub_date = None

pub_date

8 de junio de 2018


'2018-06-08'

In [54]:
p = re.compile("gFIO\[\d{1,2}\] = \\'(.*)\\';")
image_list = ['http://www.lamoncloa.gob.es' + url for url in re.findall(p, response)]
image_list

['http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618ConsejoM_10.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_06.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_08.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_11.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_05.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_10.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_12.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_14.jpg',
 'http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_03.jpg',
 '

In [55]:
p = re.compile("gFT\[\d{1,2}\] = \\'(.*)\\';")
description_list = [desc.strip().rstrip('.') for desc in re.findall(p, response)]
description_list

['Foto de familia del presidente del Gobierno, Pedro Sánchez, y las ministras y ministros de su Gabinete tras celebrar la primera reunión en La Moncloa',
 'La vicepresidenta del Gobierno y ministra de la Presidencia, Relaciones con las Cortes e Igualdad, Carmen Calvo, a su llegada a la primera reunión del Consejo de Ministras y Ministros, presidido por Pedro Sánchez en La Moncloa',
 'El ministro de Asuntos Exteriores, Unión Europea y Cooperación, Josep Borrell, a su llegada a la primera reunión del Consejo de Ministras y Ministros, presidido por Pedro Sánchez en La Moncloa',
 'La ministra de Defensa, Margarita Robles, a su llegada a la primera reunión del Consejo de Ministras y Ministros, presidido por Pedro Sánchez en La Moncloa',
 'La ministra de Hacienda, María Jesús Montero, a su llegada a la primera reunión del Consejo de Ministras y Ministros, presidido por Pedro Sánchez en La Moncloa',
 'El ministro del Interior, Fernando Grande-Marlaska, a su llegada a la primera reunión del Co

In [56]:
p = re.compile("gFA\[\d{1,2}\] = \\'(.*)\\';")
author_list = [author.strip() for author in re.findall(p, response)]
author_list

['Pool Moncloa/J.M. Cuadrado',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/J.M. Cuadrado',
 'Pool Moncloa/J.M.Cuadrado',
 'Pool Moncloa/J.M. Cuadrado',
 'Pool Moncloa/J.M.Cuadrado',
 'Pool Moncloa/ Fernando Calvo',
 'Pool Moncloa/César P. Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra',
 'Pool Moncloa/César P.Sendra']

In [57]:
truth_list = ['Pool' in author for author in author_list]
truth_list

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [58]:
len(truth_list)

31

In [59]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Galería fotográfica: ${title}]
|author=Ministerio de la Presidencia. Gobierno de España (${author})
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

vars = {
    "url": url,
    "description": None,
    "author": None,
    "date": pub_date,
    "title": title,
    "cat_string": '\n'.join(['[[Category:'+i+']]' for i in categories])
}

In [60]:
#### User input:
excluded = []

used_names = []

global_counter = 1
for i, image in enumerate(image_list):
    # If the image is excluded, skip
    if i in excluded:
        print ("Image excluded. Skipping")
        continue
    elif truth_list[i] == False:
        print ("Image from a party other than Moncloa. Skipping")
        continue        
        
    # First, the image is downloaded and stored
    image_url = quote(image.encode('utf-8'), ':/')
    print(image_url)
    image_name = '{} {:02d}.jpg'.format(title.replace(':', ' -').replace('  ', ' '), global_counter)
    global_counter += 1
    image_path = os.path.join(images_directory, image_name)
    while True:
        print ('Trying download')
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'}
        r = requests.get(image_url, headers=headers, stream=True)
        if r is not None:
            with open(image_path, 'wb') as out_file:
                shutil.copyfileobj(r.raw, out_file)
            print ('Image downloaded. Starting upload process')
            break
        else :
            print ('Still trying download')

    # If the image is already in Commons, skip
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Skipping")
        os.remove(image_path)
        global_counter -= 1
        continue

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)
        
    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    vars["description"] = description_list[i]
    vars["author"] = author_list[i]
    t = Template(template)
    _text = t.render(**vars)

    bot = UploadRobot([image_path],
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)

http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618ConsejoM_10.jpg
Trying download
Image downloaded. Starting upload process
[[commons:File:Gobierno de Pedro Sánchez en la XII Legislatura.jpg]]
Image already in commons. Skipping
http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_06.jpg
Trying download
Image downloaded. Starting upload process
[[commons:File:Gobierno de Pedro Sánchez en la XII Legislatura 02.jpg]]
Image already in commons. Skipping
http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_08.jpg
Trying download
Image downloaded. Starting upload process
[[commons:File:Josep Borrell in 2018.jpg]]
Image already in commons. Skipping
http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_11.jpg
Trying download
Image downloaded. Starting upload process
Preparing to upload image with name Gobierno de Pedr

The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La ministra de Defensa, Margarita Robles, a su llegada a la primera reunión del Consejo de Ministras y Ministros, presidido por Pedro Sánchez en La Moncloa}}
|date=2018-06-08
|source=[http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/consejoministros/Paginas/2018/080618cministros.aspx Galería fotográfica: Gobierno de Pedro Sánchez en la XII Legislatura]
|author=Ministerio de la Presidencia. Gobierno de España (Pool Moncloa/César P.Sendra)
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Palacio de la Moncloa]]
[[Category:June 2018 in Madrid]]
[[Category:Pedro Sánchez Pérez-Castejón in 2018]]
[[Category:Palacio de la Moncloa]]
[[Category:2018 in Madrid]]
Uploading file to commons:commons...
Upload successful.
Upload of Gobierno de Pedro Sánchez en la XII Legislatura 01.jpg successf

http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_05.jpg
Trying download
Image downloaded. Starting upload process
Image name (Gobierno de Pedro Sánchez en la XII Legislatura 02.jpg) already used in Commons
Preparing to upload image with name Gobierno de Pedro Sánchez en la XII Legislatura 03.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La ministra de Hacienda, María Jesús Montero, a su llegada a la primera reunión del Consejo de Ministras y Ministros, presidido por Pedro Sánchez en La Moncloa}}
|date=2018-06-08
|source=[http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/consejoministros/Paginas/2018/080618cministros.aspx Galería fotográfica: Gobierno de Pedro Sánchez en la XII Legislatura]
|author=Ministerio de la Presidencia. Gobierno de España (Pool Moncloa/César P.Sendra)
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Palacio de la Moncloa]]
[[Category:June 2018 in Madrid]]
[[Category:Pedro Sánchez Pérez-Castejón in 2018]]
[[Category:Palacio de la Moncloa]]
[[Category:2018 in Madrid]]
Uploading file to commons:commons...
Sleeping for 5.7 seconds, 2018-06-09 00:24:18
Upload successful.
Upload of Gobierno de P

http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_10.jpg
Trying download
Image downloaded. Starting upload process
[[commons:File:El ministro del Interior, Fernando Grande-Marlaska, en la Moncloa (2018).jpg]]
Image already in commons. Skipping
http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618LlegadaCM_12.jpg
Trying download
Image downloaded. Starting upload process
Image name (Gobierno de Pedro Sánchez en la XII Legislatura 03.jpg) already used in Commons
Preparing to upload image with name Gobierno de Pedro Sánchez en la XII Legislatura 04.jpg


...

The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La ministra de Trabajo, Migraciones y Seguridad Social, Magdalena Valerio, el ministro de Fomento, José Luis Ábalos, y la ministra de Hacienda, María Jesús Montero, antes de comenzar en La Moncloa la reunión del Consejo de Ministras y Ministros}}
|date=2018-06-08
|source=[http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/consejoministros/Paginas/2018/080618cministros.aspx Galería fotográfica: Gobierno de Pedro Sánchez en la XII Legislatura]
|author=Ministerio de la Presidencia. Gobierno de España (Pool Moncloa/César P.Sendra)
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Palacio de la Moncloa]]
[[Category:June 2018 in Madrid]]
[[Category:Pedro Sánchez Pérez-Castejón in 2018]]
[[Category:Palacio de la Moncloa]]
[[Category:2018 in Madrid]]
Uploading file to commons:commons...
Sle

http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618ConsejoM_02.jpg
Trying download
Image downloaded. Starting upload process
Image name (Gobierno de Pedro Sánchez en la XII Legislatura 18.jpg) already used in Commons
Preparing to upload image with name Gobierno de Pedro Sánchez en la XII Legislatura 19.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=La ministra de Política Territorial y Función Pública, Meritxell Batet, el ministro de Cultura y Deporte, Màxim Huertas, y la ministra de Sanidad, Consumo y Bienestar Social, Carmen Montón, antes de comenzar la reunión del Consejo de Ministras y Ministros, presidido por Pedro Sánchez. <br>}}
|date=2018-06-08
|source=[http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/consejoministros/Paginas/2018/080618cministros.aspx Galería fotográfica: Gobierno de Pedro Sánchez en la XII Legislatura]
|author=Ministerio de la Presidencia. Gobierno de España (Pool Moncloa/César P.Sendra)
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Palacio de la Moncloa]]
[[Category:June 2018 in Madrid]]
[[Category:Pedro Sánchez Pérez-Castejón in 2018]]
[[Category:Palacio de la Moncloa]]
[[Category:2018 in Mad

http://www.lamoncloa.gob.es/multimedia/fotos/consejoministros/PublishingImages/2018/080618ConsejoM_01.jpg
Trying download
Image downloaded. Starting upload process
Image name (Gobierno de Pedro Sánchez en la XII Legislatura 19.jpg) already used in Commons
Preparing to upload image with name Gobierno de Pedro Sánchez en la XII Legislatura 20.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=El ministro de Ciencia, Innovación y Universidades, Pedro Duque, y la ministra de Economía, Nadia Calviño, antes de comenzar la reunión del Consejo de Ministras y Ministros, presidido por Pedro Sánchez}}
|date=2018-06-08
|source=[http://www.lamoncloa.gob.es/multimedia/galeriasfotograficas/consejoministros/Paginas/2018/080618cministros.aspx Galería fotográfica: Gobierno de Pedro Sánchez en la XII Legislatura]
|author=Ministerio de la Presidencia. Gobierno de España (Pool Moncloa/César P.Sendra)
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Palacio de la Moncloa]]
[[Category:June 2018 in Madrid]]
[[Category:Pedro Sánchez Pérez-Castejón in 2018]]
[[Category:Palacio de la Moncloa]]
[[Category:2018 in Madrid]]
Uploading file to commons:commons...
Sleeping for 5.8 seconds, 2018-06-09 00:27:08
